## DenseNet 

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
gr = 32  #growth_rate

In [3]:
def DeneseLayer(data , name):
    with tf.variable_scope( name , reuse=tf.AUTO_REUSE ) as scope:
        out = tf.layers.batch_normalization(inputs= data )
        out = tf.nn.relu(out)
        out = tf.layers.conv2d(inputs= out , filters=4*gr ,
        kernel_size = [1,1] , use_bias=False)
        out = tf.layers.batch_normalization(inputs= out )
        out = tf.nn.relu(out)
        out = tf.layers.conv2d(inputs= out , filters= gr ,
        kernel_size = [3,3] , use_bias=False , padding = 'SAME')
        data = tf.concat([out,data] , axis = 3 )
    return data


In [4]:
def transition(data , name):
    with tf.variable_scope( name , reuse=tf.AUTO_REUSE ) as scope:
        size = data.shape[3]
        data = tf.layers.batch_normalization(inputs=data)
        data = tf.nn.relu(data)
        data = tf.layers.conv2d(inputs= data , filters= size ,
        kernel_size=[1,1] , use_bias= False ,
        activation= tf.nn.relu , padding = 'SAME'  )
        data = tf.layers.average_pooling2d( inputs = data ,
        pool_size=[2,2] , strides = (2,2) , padding = 'SAME'  )
    return data


In [5]:
inp = tf.random_uniform([1 , 224 , 224 , 3])

In [6]:
def model(data):
    
    out = tf.layers.conv2d(
    inputs = data ,
    filters=2*gr , 
    kernel_size=[7,7] ,
    strides=(2,2) ,
    padding = 'SAME' , 
    use_bias=False , 
    name = 'Conv1' ,
    reuse = tf.AUTO_REUSE)
    
    out = tf.layers.max_pooling2d(
    inputs=out,
    pool_size=[3,3],
    strides=(2,2),
    padding='SAME',
    name = 'Pool1' )
        
    with tf.variable_scope('Block1') as scope:
        for i in range (6) :
            out = DeneseLayer(out , 'Block1' + 'dense_layer.{}'.format(i))

        out = transition (out , 'Block1')
        
    print(out)
            
    with tf.variable_scope('Block2') as scope:
        for i in range (12):
             out = DeneseLayer(out , 'Block2' + 'dense_layer_{}'.format(i))

        out = transition (out , 'Block2')
        
    print(out)
                
    with tf.variable_scope('Block3') as scope:
        for i in range (24) :
            out = DeneseLayer(out , 'Block3' + 'dense_layer_{}'.format(i))

        out = transition (out , 'Block3')
        
    print(out)
    
        
    with tf.variable_scope('Block4') as scope:
        for i in range (16) :
            out = DeneseLayer(out , 'Block4' + 'dense_layer_{}'.format(i))
    
    print(out)
    
    
    out = tf.keras.layers.GlobalAveragePooling2D()(out)
    
    out = tf.layers.dense(
    inputs= out ,
    units = 1000 ,
    reuse = tf.AUTO_REUSE ,
    name = 'out_layer')
        
    return out

In [7]:
output = model(inp)

Tensor("Block1/Block1/average_pooling2d/AvgPool:0", shape=(1, 28, 28, 256), dtype=float32)
Tensor("Block2/Block2/average_pooling2d/AvgPool:0", shape=(1, 14, 14, 640), dtype=float32)
Tensor("Block3/Block3/average_pooling2d/AvgPool:0", shape=(1, 7, 7, 1408), dtype=float32)
Tensor("Block4/Block4dense_layer_15/concat:0", shape=(1, 7, 7, 1920), dtype=float32)
